In [1]:
import pandas as pd
import json
from pathlib import Path

In [2]:
CONSTS = {
    "SHEET_UNITS_POSITION": "https://docs.google.com/spreadsheets/d/e/2PACX-1vS006XXuC5badxZzCbYily_0PiSSp3o4QeLWAO4clkziXcD0MBaHEmD62kACdVVqq4YGsSaR1l9ijT4/pub?gid=2022893692&single=true&output=csv",
    "SHEET_UNITS_DESCRIPTION": "https://docs.google.com/spreadsheets/d/e/2PACX-1vS006XXuC5badxZzCbYily_0PiSSp3o4QeLWAO4clkziXcD0MBaHEmD62kACdVVqq4YGsSaR1l9ijT4/pub?gid=187612759&single=true&output=csv",
    "export_folder": Path('./data/'),
    "export_folder_website": Path('./website_tmp')
}

In [21]:
df = pd.read_csv(CONSTS['SHEET_UNITS_POSITION'])

df_description = pd.read_csv(CONSTS['SHEET_UNITS_DESCRIPTION'])

# Merge
df = df.merge(df_description, how='left', on='unit')
if len(df[df.icon.isna()]) > 0:
    print(df[df.icon.isna()])
    raise ValueError("🤬 Unit Position withouth corresponding Unit in 'units_description'")

# Sort values
df = df.sort_values(['date', 'unitnumber'])

data = {
        "type": "FeatureCollection",
        "features": []
    }

# Fill nas
df['subordinate_to'] = df['subordinate_to'].fillna('')
df['unit'] = df['unit'].fillna('')

# Convert date
try:
    df['date'] = pd.to_datetime(df['date'])
except:
    raise ValueError("🤬 Field 'date' could not be converted to DateTime. Please check Date column")

# Convert Lat Lng
try:
    df['lat'] = df['location'].apply(lambda x: x.replace(' ', '').split(',')[0])
    df['lng'] = df['location'].apply(lambda x: x.replace(' ', '').split(',')[1])
except:
    raise ValueError("🤬 Could not convert 'Location? into Lat Lng. Probably empty or invalid input")

df = df[['date', 'lat', 'lng', 'icon', 'type', 'strength', 'strength_in_btg_text', 'strength_in_btg_number', 'unit', 'unitnumber', 'subordinate_to']]

# add Grouper
df['_group'] = df.apply(lambda row: "%s-%s-%s" % (row['date'], row['lat'], row['lng']), axis=1)
df = df.groupby(['_group', 'unit']).first()


In [34]:
df.head(2)

,,date,lat,lng,icon,type,strength,strength_in_btg_text,strength_in_btg_number,unitnumber,subordinate_to
_group,unit,,,,,,,,,,
2022-03-31 00:00:00-46.631259-32.439439,20th Guards Motorized Rifle Division,2022-03-31,46.631259,32.439439,SHGPUCIZ--AI,Headquarters,Division,0,0,20,22nd Army Corps
2022-03-31 00:00:00-46.677683-32.508789,22nd Army Corps,2022-03-31,46.677683,32.508789,SHGPUCIZ--AJ,Headquarters,Corps,0,0,22,Black Sea Fleet


In [43]:
for i, new_df in df.groupby(level=0):
    break

new_df.reset_index()

,_group,unit,date,lat,lng,icon,type,strength,strength_in_btg_text,strength_in_btg_number,unitnumber,subordinate_to
0,2022-03-31 00:00:00-46.631259-32.439439,20th Guards Motorized Rifle Division,2022-03-31,46.631259,32.439439,SHGPUCIZ--AI,Headquarters,Division,0,0,20,22nd Army Corps


In [45]:
counter = 0
for i, new_df in df.groupby(level=0):
    try:
        units = []
        icons = []

        new_df = new_df.reset_index()

        for j, row in new_df.iterrows():

            units.append({
                    "icon": row['icon'],
                    "type": row['type'],
                    "strength": row['strength'].strip(),
                    "strength_in_btg_number": row['strength_in_btg_number'],
                    "strength_in_btg_text": row['strength_in_btg_text'],
                    "unit": row['unit'].strip(),
                    "unitnumber": row['unitnumber'] if pd.notna(row['unitnumber']) else None,
                    "subordinate_to": row['subordinate_to'].strip(),
                })

            icons.append(row['icon'])

        data['features'].append({
            "type": "Feature",
            "id": counter + 1,
            "properties": {
                "date": new_df.iloc[0]['date'].strftime('%Y-%m-%d'),
                "type": "single",
                "icon": {
                    "type": 'single',
                    "icons": icons
                },
                "units": units
            },
            "geometry": {
                "type": "Point",
                "coordinates": [float(new_df.iloc[0]['lng']), float(new_df.iloc[0]['lat'])]
            }
        })

        counter += 1
    except Exception as e:
        print("🥵 Something wroing in row %s in Sheet troops" % (i + 1))
        print(list(row))
        print("ℹ️ Errormessage: %s" % e)
        pass
        break
    

In [46]:
    
# Export website
json.dump(data, open(CONSTS['export_folder_website'] / Path('./units.json'), 'w', encoding='UTF-8'), ensure_ascii=False)

FileNotFoundError: [Errno 2] No such file or directory: 'website_tmp/units.json'

In [50]:
x = 'SFGPU-----A1'


'SFGPU------1'